<a href="https://colab.research.google.com/github/yingwang/genai_image/blob/main/GFPGAN_vs_RealESRGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install basicsr facexlib gfpgan
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
!pip install -r requirements.txt
!python setup.py develop
%cd ..

In [ ]:
# !rm -rf Real-ESRGAN

# !git clone https://github.com/xinntao/Real-ESRGAN.git
# %cd Real-ESRGAN

# !pip install -r requirements.txt
# !pip install -e .

# import sys
# sys.path.insert(0, "/content/Real-ESRGAN")

# try:
#     from realesrgan import RealESRGANer
#     print("Loaded RealESRGANer!")
# except ImportError as e:
#     print("Fail to load:", e)

In [ ]:
!python -m pip install torch==2.0.1 torchvision==0.15.2 --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from gfpgan import GFPGANer
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def initialize_models():
    # GFPGAN for face restoration
    gfpgan = GFPGANer(
        model_path='https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth',
        upscale=1,
        arch='clean',
        channel_multiplier=2,
        bg_upsampler=None
    )

    # ESRGAN for general image enhancement
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32)
    esrgan = RealESRGANer(
        scale=4,
        model_path='https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth',
        model=model,
        tile=400,
        tile_pad=10,
        pre_pad=0,
        half=False
    )
    return gfpgan, esrgan

gfpgan, esrgan = initialize_models()

input_dir = '/content/drive/MyDrive/input_images'
output_dir = '/content/drive/MyDrive/output_images'
os.makedirs(output_dir, exist_ok=True)

image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
print(f"Found {len(image_files)} images to process")

In [ ]:
def process_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error reading {img_path}")
        return None, None

    _, _, gfpgan_result = gfpgan.enhance(
        img,
        has_aligned=False,
        only_center_face=False,
        paste_back=True
    )

    esrgan_result, _ = esrgan.enhance(img, outscale=4)

    # Resize ESRGAN result to match GFPGAN size
    esrgan_resized = cv2.resize(esrgan_result, (gfpgan_result.shape[1], gfpgan_result.shape[0]))

    return gfpgan_result, esrgan_resized

for img_file in image_files:
    try:
        input_path = os.path.join(input_dir, img_file)
        print(f"\nProcessing {img_file}...")

        gfp_result, esr_result = process_image(input_path)

        if gfp_result is not None and esr_result is not None:
            comparison = np.hstack((gfp_result, esr_result))

            base_name = os.path.splitext(img_file)[0]
            cv2.imwrite(os.path.join(output_dir, f"{base_name}_GFPGAN.png"), gfp_result)
            cv2.imwrite(os.path.join(output_dir, f"{base_name}_ESRGAN.png"), esr_result)
            cv2.imwrite(os.path.join(output_dir, f"{base_name}_comparison.png"), comparison)

            print("Original vs GFPGAN vs ESRGAN (Left to Right)")
            display_img = np.hstack((cv2.resize(cv2.imread(input_path), (256, 256)),
                                   cv2.resize(gfp_result, (256, 256)),
                                   cv2.resize(esr_result, (256, 256))))
            cv2_imshow(display_img)

    except Exception as e:
        print(f"Error processing {img_file}: {str(e)}")

print("\nProcessing complete! Check your output folder in Google Drive")